# pseudo code 

In [ ]:
다른 버스들과 마찬가지로 1112번도 사색에서 도착정보(사색의 광장에 버스가 도착하기 전까지 몇분 남았는지)를 수집할 수 없다. 
그래서 생대역의 도착 정보를 이용할 것이다.
생대역에서 도착정보 없음 계속 뜨다가 처음 정상처리이 뜨는 순간이 사색에 버스가 도착했다는 뜻이다. 그때마다 시간을 기록할 것이다. 
그리고 사색의 광장에 버스가 도착했을 당시에, 운행중인 1112번 버스들의 위치(정류장)도 함께 수집해서 규칙성이 존재하는지 볼 것이다. 
이 코드의 특징:
1112번외에도 다른 버스들도 똑같이 수집 가능하고,
불연속 점을 도출하기 위해 초마다 api 호출을 안해도 되고, 
[사색 도착시간,운행중인 ㅇㅇ버스들의 현 위치]를 수집할 수 있다.

past=없다
1분마다 요청을 한다.
    result=도착정보가 있다 or 없다
    if past!=result:
        if result==도착정보 있다
            처음으로 '있다'가 나왔다. '현시간=사색 도착 시간'을 저장한다. 
    past=result 값 임시 저장

# 실제코드

In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time

bus_time=[]
p5100 ={'serviceKey' : 'Jw+++HTzL3V8lfZPcQPb5jB5Rae2NH3BYowjBY+cgMIA1yDcwJNfmtfO+E+O+whPsGCGbJmmpf/X904L4uwGIw==',
             'routeId':'200000115','stationId':'228000704'}#5100 생과대
p1112 ={'serviceKey' : 'Jw+++HTzL3V8lfZPcQPb5jB5Rae2NH3BYowjBY+cgMIA1yDcwJNfmtfO+E+O+whPsGCGbJmmpf/X904L4uwGIw==',
         'routeId':'234000016','stationId':'228000704'}#1112 생과대

def repet(params): 
    url = 'http://apis.data.go.kr/6410000/busarrivalservice/getBusArrivalItem'
    response = requests.get(url, params=params);
    soup=BeautifulSoup(response.text,'xml')
    return soup

def current_buses(): # 사색의 광장에 버스가 도착했을 때, 운행중인 같은 노선의 버스들의 위치(정류장)를 수집  
    url = 'http://apis.data.go.kr/6410000/buslocationservice/getBusLocationList'
    params ={'serviceKey' : 'Jw+++HTzL3V8lfZPcQPb5jB5Rae2NH3BYowjBY+cgMIA1yDcwJNfmtfO+E+O+whPsGCGbJmmpf/X904L4uwGIw==',
              'routeId':'234000016'}#1112번 노선id
    response=requests.get(url,params=params)
    soup=BeautifulSoup(response.text,'xml')
    locations=sorted([int(x.stationSeq.text) for x in soup.find_all('busLocationList')])
    return locations


In [27]:
past=4
for x in range(10): # 10분간
    soup=repet(p1112)
    try:
        result=int(soup.resultCode.text)
        if past!=result:
            if result==0:
                bus_time.append([soup.queryTime.text,current_buses()])
        past=result
        print(x,"번째 시도: ",bus_time)
        time.sleep(60)
    except Exception as e:
        continue
df=pd.DataFrame(bus_time,columns=['사색 도착 시간','운행중인 다른 버스들의 위치'])
df

0 번째 시도:  []
1 번째 시도:  []
2 번째 시도:  []
3 번째 시도:  []
4 번째 시도:  []
5 번째 시도:  []
6 번째 시도:  []
7 번째 시도:  []
8 번째 시도:  [['2022-07-02 13:52:26.432', [2, 15, 19, 25, 32, 39, 45, 52, 61, 68]]]
9 번째 시도:  [['2022-07-02 13:52:26.432', [2, 15, 19, 25, 32, 39, 45, 52, 61, 68]]]


,사색 도착 시간,운행중인 다른 버스들의 위치
0,2022-07-02 13:52:26.432,"[2, 15, 19, 25, 32, 39, 45, 52, 61, 68]"


In [30]:
# string 값인 시간을 datetime객체로 바꾼다. 
df['사색 도착 시간'] = df['사색 도착 시간'].apply(pd.to_datetime)
df

,사색 도착 시간,운행중인 다른 버스들의 위치
0,2022-07-02 13:52:26.432,"[2, 15, 19, 25, 32, 39, 45, 52, 61, 68]"


# 오늘은 주말이라 버스가 너무 없지만 평일 데이터가 점점 쌓이면 규칙을 발견할 수 있을 것 같다.